In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)


memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is jinwook")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is jinwook
    You:


> Finished chain.


'Hello Jinwook! How can I assist you today?'

In [2]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is jinwook
AI: Hello Jinwook! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


"That's great! Seoul is a vibrant and bustling city. How can I assist you today, Jinwook?"

In [3]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is jinwook
AI: Hello Jinwook! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant and bustling city. How can I assist you today, Jinwook?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Jinwook.'

In [4]:
memory.load_memory_variables({})

{'chat_history': "Human: My name is jinwook\nAI: Hello Jinwook! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great! Seoul is a vibrant and bustling city. How can I assist you today, Jinwook?\nHuman: What is my name?\nAI: Your name is Jinwook."}